In [7]:
import pandas as pd
import xml.etree.ElementTree as ET

In [114]:
TRAIN = False

if TRAIN:
    INPUT_XML_FILE = './dataset/train.xml'
    SCORES_OUT_FILE = './cache/train.parquet'
else:
    INPUT_XML_FILE = './dataset/test.xml'
    SCORES_OUT_FILE = './cache/test.parquet'

In [115]:
tree = ET.parse(INPUT_XML_FILE)
score_columns = ['beeline', 'mts', 'megafon', 'tele2', 'rostelecom', 'komstar', 'skylink']
columns = ['id', 'text'] + score_columns
df = pd.DataFrame.from_records(
    [[record.find(f"column[@name='{attr}']").text for attr in columns] for record in tree.getroot().find('database').iter('table')],
    columns=columns,
)
df['id'] = df['id'].astype('uint16')
df['text'] = df['text'].astype('string')
df.set_index('id', inplace=True)
df.head()

,text,beeline,mts,megafon,tele2,rostelecom,komstar,skylink
id,,,,,,,,
1,RT @vestihitech: Лайфхак от Билайн: как зарабо...,-1,NULL,NULL,NULL,NULL,NULL,NULL
2,RT @yablochno_com: iPhone 5S и 5C заработали в...,1,NULL,NULL,NULL,NULL,NULL,NULL
3,RT @HAPKOTNKN: «Билайн» включил LTE для владел...,1,NULL,NULL,NULL,NULL,NULL,NULL
4,"@domrist =&gt; Указанную информацию, пожалуйст...",0,NULL,NULL,NULL,NULL,NULL,NULL
5,Ура! Теперь заработает#Lte#4g#beeline#nanosim#...,1,NULL,NULL,NULL,NULL,NULL,NULL


In [116]:
len(df)

3845

In [117]:
known_scores = {
    '0': 0,
    '-1': -1,
    '1': 1,
}

def merge_scores(row):
    score = next(filter(lambda x: x != 'NULL', row[score_columns]))
    row['score'] = known_scores[score] if score in known_scores else None
    return row

score_df = df.apply(merge_scores, axis=1).drop(score_columns, axis=1).dropna(how='any', axis=0)
score_df['score'] = pd.Categorical(score_df['score'].astype('int'))
score_df.head()

,text,score
id,,
1,RT @vestihitech: Лайфхак от Билайн: как зарабо...,-1
2,RT @yablochno_com: iPhone 5S и 5C заработали в...,1
3,RT @HAPKOTNKN: «Билайн» включил LTE для владел...,1
4,"@domrist =&gt; Указанную информацию, пожалуйст...",0
5,Ура! Теперь заработает#Lte#4g#beeline#nanosim#...,1


In [118]:
score_df.to_parquet(SCORES_OUT_FILE)

In [119]:
score_df['score'].value_counts()

0     2595
-1     879
1      371
Name: score, dtype: int64